<a href="https://colab.research.google.com/github/hdemma/hdemma.github.io/blob/master/Prediction_Script_Diesel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

from keras.models import load_model
import math

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from keras.models import Sequential,load_model

from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


# Load Data Set

In [3]:
Vehicle_Type = 'Diesel_Vehicles'
Folder = '..Dataset/SampleData'

url = 'https://raw.githubusercontent.com/hdemma/hdemma.github.io/develop/Dataset/Diesel_Vehicles/Diesel_Vehicles_Final_Training_Samples.csv'
df = pd.read_csv(url)
print(len(df))

54741


# Processsing Inputs and Labels

In [4]:
df = df.values.tolist()
DataSet = np.array(df,dtype=float)

print('*******************************')

Tupple = np.shape(DataSet)                  #34011 Segments x 18 Features
Number_of_Records = Tupple[0]
Number_of_Features = Tupple[1]

print(f'Number_of_Records = {Number_of_Records}')
print(f'Number_of_Features = {Number_of_Features}')


#Processing Sample_Inputs without Target Feature

X = np.zeros((Number_of_Records, Number_of_Features - 1))
for t in range(Number_of_Records):
    X[t] = DataSet[t, :(Number_of_Features-1)]

print(f'Input_Size = {np.shape(X)}')

#Processings Sample_Lables with Target feature ('Fuel_Consumed')

y = np.zeros((Number_of_Records,))
for t in range(Number_of_Records):
    y[t] = DataSet[t][Number_of_Features-1]

print(f'Label_Size = {np.shape(y)}')


print('*******************************')

*******************************
Number_of_Records = 54741
Number_of_Features = 23
Input_Size = (54741, 22)
Label_Size = (54741,)
*******************************


# Random Train-Test Split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

print(f'Training_Input = {np.shape(X_train)}')
print(f'Training_Labels = {np.shape(y_train)}')

print(f'Testing_inputs = {np.shape(X_test)}')
print(f'Testing_Labels = {np.shape(y_test)}')

Training_Input = (43792, 22)
Training_Labels = (43792,)
Testing_inputs = (10949, 22)
Testing_Labels = (10949,)


# Removing Time_Needed Feature

In [6]:
# Saving Time Needed for predicting for samples with longer intervals
Time_needed = []

for i in X_test:
    Time_needed.append(i[1])

print(np.shape(Time_needed))

#Deleting Time_Needed before Training

X_train = np.delete(X_train, 1, 1)
X_test = np.delete(X_test, 1, 1)

print(np.shape(X_train))
print(np.shape(y_train))

T = np.shape(X_test) #For Plotting

print(T)

Length_of_TestData = T[0]

(10949,)
(43792, 21)
(43792,)
(10949, 21)


# ANN Model - Feed Forward

In [0]:
def SimpleFeedForward_Model():

  if Train:

    FF_model = Sequential()

    FF_model.add(Dense(400,  input_dim=Number_of_Features-2, activation="sigmoid"))
    FF_model.add(Dense(200,  activation="sigmoid"))
    FF_model.add(Dense(100, activation="sigmoid"))
    FF_model.add(Dense(50, activation="sigmoid"))
    FF_model.add(Dense(25, activation="sigmoid"))
    FF_model.add(Dense(1, activation = "linear"))

    optimizer = keras.optimizers.SGD(lr=0.1, momentum=0.00001)

    FF_model.compile(loss="mean_squared_error", optimizer = 'adam', metrics=['mse', 'mae'])

    FF_model = f'FF_model_{epoch:03d}_{loss:03f}_{val_loss:03f}.h5'
    checkpoint = ModelCheckpoint(f'{Folder}/FF_model_{epoch:03d}_{loss:03f}_{val_loss:03f}.h5', verbose=1, monitor='val_loss',
                                 save_best_only=True, mode='auto')
    
    history = FF_model.fit(X_train, y_train,
        shuffle=True,
        epochs=600,
        batch_size=128,
        # validation_split=0.05,
        validation_data=(X_test,y_test),
        callbacks=[checkpoint],
        verbose=2)



    print(history.history.keys())

    plt.plot(history.history['mean_squared_error'])
    plt.plot(history.history['val_mean_squared_error'])
    plt.title('mean_squared_error')
    plt.xlabel('Iteration')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    plt.plot(history.history['mean_absolute_error'])
    plt.plot(history.history['val_mean_absolute_error'])
    plt.title('mean_absolute_error')
    plt.xlabel('Iteration')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Iteration')
    plt.legend(['Train', 'Test'], loc='upper left')

    plt.show()

    print("Saved model to disk")

    y_pred = FF_model.predict(X_test, verbose=2, batch_size=128)

    return y_pred

  else:
    #### load model

    url = 'https://github.com/hdemma/hdemma.github.io/blob/develop/Dataset/PredictionModels/Diesel/FF_model-765-0.000677-0.000736.h5'

    FF_model = load_model(url)

    print("MSE\t\tMAE")

    scores = FF_model.evaluate(X_train, y_train, verbose=2)
    print("Tarining Set Loss: {}".format(scores))

    scores = FF_model.evaluate(X_test, y_test, verbose=2)
    print("Testing Set Loss: {}".format(scores))

    y_pred = FF_model.predict(X_test, verbose=2, batch_size=128)

    return y_pred

# Decision Tree Model

In [0]:
def DecisionTree_Model():
  
  # filename = f'{Folder}/PredictionModels/Diesel/finalized_DT_model.sav'
  if Train:
    DT_regressor = DecisionTreeRegressor(max_depth=Number_of_Features,
                                         splitter='best',
                                         criterion="mse",
                                         min_samples_leaf=3)
    
    # fit the DT regressor
    DT_regressor.fit(X_train, y_train)
    
    y_pred = DT_regressor.predict(X_train)
    
    MSE = np.sum(((y_train - y_pred) ** 2) / len(y_test))
    print(f'MSE on Train = {MSE}')
    
    y_pred = DT_regressor.predict(X_test)
    
    MSE = np.sum(((y_test - y_pred) ** 2) / len(y_test))
    print(f'MSE on Test = {MSE}')
    
    RMSE = np.sqrt(np.sum(((y_test - y_pred) ** 2) / len(y_test)))
    print(f'RMSE = {RMSE}')
    
    MAE = np.sqrt(np.sum(abs(y_test - y_pred) / len(y_test)))
    print(f'MAE = {MAE}')
    
    score = DT_regressor.score(X_test, y_test)
    print(f'Score = {score}\n\n')

    ### save the model to disk
    
    pickle.dump(DT_regressor, open(filename, 'wb'))

    return y_pred

  else:

    ###load the model from disk
    url='https://github.com/hdemma/hdemma.github.io/blob/develop/Dataset/PredictionModels/Diesel/finalized_DT_model.sav'
    print('\nError')
    model = !wget url
    print(model)
    loaded_model = pickle.load(open(url, 'rb'))
    y_pred = loaded_model.predict(X_test)
    score = loaded_model.score(X_test,y_test)
    print(f'R^2 Score = {score}')

    return y_pred

# Linear Regression Model

In [0]:
def LinearRegression_Model():

  filename = f'{Folder}/PredictionModels/Diesel/finalized_LR_model.sav'

  if Train:
    regression_model = LinearRegression(fit_intercept=True, normalize=True)
    
    # fit the regressor
    regression_model.fit(X_train, y_train)
    
    # regression coefficients
    print('Coefficients for each feature:')
    print(regression_model.coef_)
    
    R_Square = regression_model.score(X_test, y_test)  # 1 is perfect prediction
    print(f'R_Square = {R_Square}')
    
    y_pred = regression_model.predict(X_train)
    
    regression_model_mse = mean_squared_error(y_train, y_pred)
    regression_model_mae = mean_absolute_error(y_train, y_pred)
    
    print(f'regression_model_mse on Train = {regression_model_mse}')
    print(f'regression_model_mae on Train = {regression_model_mae}')
    
    y_pred = regression_model.predict(X_test)
    
    regression_model_mse = mean_squared_error(y_test, y_pred)
    regression_model_mae = mean_absolute_error(y_test, y_pred)
    
    print(f'regression_model_mse on Test = {regression_model_mse}')
    print(f'regression_model_mae on test = {regression_model_mae}')


    ### save the model to disk
    pickle.dump(regression_model, open(filename, 'wb'))
    
    return y_pred

  else:

    ###load the model from disk
    loaded_model = pickle.load(open(filename, 'rb'))
    y_pred = loaded_model.predict(X_test)
    score = loaded_model.score(X_test,y_test)
    print(f'R^2 Score = {score}')

    return y_pred

# Plotting Predicted vs Actual

In [0]:
def Plotting_Pred_Actual(Model_Name,Actual_Values,PredictedValues,Length_of_TestData,Count):

    #Each iteration will plot results for 25 samples.
    
    initial = 0
    final = 25
    for initial in range(0,Length_of_TestData+1,25):
        Samples = list(range(initial,final,1))
    
        plt.grid(color='b', linestyle='--', linewidth=0.1)
        plt.plot(PredictedValues[initial:final],'*',color = 'k')
        plt.plot(Actual_Values[initial:final], '--',marker = 'o',color = 'r')
        plt.xlabel(f'Samples from {initial} to {final-1}',size = 20)#,weight='bold')
        xi = [i for i in range(0,len(Samples),1)]
        plt.xticks(xi,Samples,rotation=90)
    
        plt.ylabel('Energy Consumption',size=20)#,weight='bold')
        plt.title(f'Predicted Energy Consumption vs Actual Energy Consumption\n{Model_Name}',size=20)#,weight='bold')
        plt.legend(('Predicted Value', 'Actual Value'),
                   loc='upper right',fontsize = 'x-large')
    
    
        plt.show()

        final = final + 25
        
        if final > Length_of_TestData:
            final = Length_of_TestData
        
        if final>Count:  #will plot for the first 50 Samples
            break

# Saving Results in files

In [0]:
def Save_Results_to_Files(Model_Name,Actual_Values,PredictedValues,Time_needed):

    Abs_Loss = []
    for i in range(0,len(Actual_Values)):
        Abs_Loss.append(abs(PredictedValues[i] - Actual_Values[i]))

    Abs_Loss = np.array(Abs_Loss)
    PredictedValues = np.array(PredictedValues)
    Actual_Values=np.array(Actual_Values)
    Loss_in_Percentage = []



    Predicted_Fuel_Consumption = pd.DataFrame({"Predicted_Values": PredictedValues,
                                                "Actual_Values":Actual_Values,
                                               "Abs_Loss":Abs_Loss,
                                                "Time_needed":Time_needed})

    CSV_Name_Loss = f'{Model_Name}_Predicted_Fuel_Consumption_for_{Vehicle_Type}.csv'
    Predicted_Fuel_Consumption.to_csv(f'{Folder}/Results/Diesel/{CSV_Name_Loss}',index=False)
    print('\n\n***********CSV Saved************\n')

# Prediction with Different Network

In [26]:
Models = ['DecisionTree','SimpleFeedForward','LinearRegression']

Actual_Values = y_test
Train = False

for Model_Name in Models:

    if Model_Name == 'LinearRegression':
        print('\n\n\n*************Executing LR*************\n\n')
        PredictedValues = LinearRegression_Model()

    if Model_Name == 'DecisionTree':
        print('\n\n\n*************Executing DT*************\n\n')
        PredictedValues = DecisionTree_Model()

    if Model_Name == 'SimpleFeedForward':
        print('\n\n\n*************Executing FF*************\n\n')
        # number_of_iteration = 800
        PredictedValues_FF = SimpleFeedForward_Model()
        Model_Name = 'SimpleFeedForward'
        PredictedValues = []
        for i in PredictedValues_FF:
            for j in i:
                PredictedValues.append(j)
        print(len(PredictedValues))

    ## Plot outputs
    Count = 50             #Indicates Plotting for how many samples
    Plotting_Pred_Actual(Model_Name,Actual_Values,PredictedValues,Length_of_TestData,Count)


    ## Creating a CSV with the predicted values

    Save_Results_to_Files(Model_Name,Actual_Values,PredictedValues,Time_needed)





*************Executing DT*************



Error
['--2020-05-20 18:37:20--  http://url/', 'Resolving url (url)... failed: Name or service not known.', 'wget: unable to resolve host address ‘url’']


FileNotFoundError: ignored